

## Efficient Scraping Using Google Colab and Data Slicing

Having acquired a list of company names and their corresponding links to the NSW portal in `supplier_data_agriculture.xlsx`, we can now utilize this data to scrape additional details from each company's official profile on the NSW portal. This includes scraping for the company's official website and Australian Business Number (ABN).

### Step 1: Setup in Google Colab Notebooks

1. **Open New Notebooks in Google Colab:**
   - Use separate notebooks to handle different segments of the data for parallel processing.

2. **Load the Excel File and Libraries:**
   - Import the `supplier_data_agriculture.xlsx` file into each notebook.
   - Import necessary libraries such as `pandas` for data manipulation, `selenium` for web scraping, and `re` for regular expressions.

### Step 2: Data Slicing for Parallel Processing

- **Divide the Dataset:**
  - If there are, for instance, 300 rows in `supplier_data_agriculture.xlsx`, divide them for processing in separate notebooks:
    - Notebook 1: Rows 1-100
    - Notebook 2: Rows 101-200
    - Notebook 3: Rows 201-300
  - Adjust the slicing in the script accordingly.

### Step 3: Scraping Company Websites and ABN

1. **Scraping Company Websites:**
   - Use the portal links from the sliced data segment in each notebook.
   - Implement the `get_supplier_website` function to navigate to each company's page and scrape the URL of the "Go to supplier website" hyperlink.
   - Store this URL in a new column in your DataFrame.

2. **Scraping ABN Details:**
   - Continue using the same portal links to scrape the company's ABN.
   - Implement the `get_supplier_abn` function, which uses a regular expression to find and extract the 11-digit ABN formatted as 2-3-3-3.
   - Add this ABN data to another column in your DataFrame.

### Step 4: Executing the Scripts

- In each notebook, after adjusting the data slicing, run the scraping scripts:
  - The first script scrapes and prints the company's website URL for each supplier.
  - The second script scrapes and prints the ABN for each supplier.


In [ ]:
##FOR WEBSITES

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

def get_supplier_website(supplier_url):
    driver.get(supplier_url)

    try:
        website_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "Go to supplier website"))
        )
        return website_link.get_attribute('href')
    except:
        return "Website link not found"

# read excel file and create a list of supplier URLs. Note this sheet is in Colab cloud so no path is required
excel_file_path = 'supplier_data_market.xlsx'  #this is the datasheet containing company name and url (not company website)
df = pd.read_excel(excel_file_path)
supplier_urls = df['Supplier URL'].tolist()

#DO THE SLICING HERE
supplier_urls = supplier_urls[:100]

# Loop through each supplier URL
for url in supplier_urls:
    website_url = get_supplier_website(url)
    print(url + " - " + website_url)

# Quit the driver when done
driver.quit()

In [ ]:
###FOR ABN


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

def get_supplier_abn(supplier_url):
    driver.get(supplier_url)

    try:
        # This is to get all the hyperlinks present in the page
        hyperlinks = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "a"))
        )

        # ABN is in the sequence of 2,3,3,3 (total 11 digits)
        abn_pattern = re.compile(r'\d{2} \d{3} \d{3} \d{3}')  # Pattern for ABN

        for link in hyperlinks:
            if abn_pattern.search(link.text):
                return link.text

        return "ABN not found"
    except:
        return "ABN not found"

excel_file_path = 'supplier_data_agriculture.xlsx'
df = pd.read_excel(excel_file_path)
supplier_urls = df['Supplier URL'].tolist()[:100]  # Slice to the first 100 rows

for url in supplier_urls:
    abn = get_supplier_abn(url)
    print(url, "-  ", abn)

driver.quit()



## Managing Scraping Results for Efficiency

Once the scraping process is complete, the results will be displayed in the format of `"NSW Portal Link" - "Website/ABN"`. While saving these results directly to an Excel file is an option, it can be time-consuming, especially if you're dealing with a large amount of data. An efficient alternative is to utilize a two-step copy-paste method involving a text editor (like Notepad or a Notebook).

### Step-by-Step Guide to Efficiently Manage Results:

1. **Copy the Output:**
   - After the scraping script finishes execution, you will have a list of results displayed in your Google Colab notebook.
   - Select and copy the entire output from the notebook.

2. **Paste into a Text Editor:**
   - Open a simple text editor (like Notepad) or use another notebook.
   - Paste the copied results into the text editor. 
   - This step ensures that the data is formatted correctly for the next stage.

3. **Copy from the Text Editor:**
   - Once pasted into the text editor, select and copy the data again. This step is crucial as it ensures that the data is in a format that will be correctly recognized by Excel. 

4. **Paste into Excel:**
   - Open your existing `supplier_data_agriculture.xlsx` file.
   - Paste the data into a new column. 
   - Since each entry is in the format `"NSW Portal Link" - "Website/ABN"`, we can neatly align it with the existing rows.

5. **Matching Data with Existing Rows:**
   - The inclusion of the "NSW Portal Link" in the output is intentional and serves an important purpose. 
   - It allows you to easily match the scraped website URLs or ABNs with the correct rows in your main sheet, based on the NSW Portal Link.

### Advantages of This Method:

- **Efficiency:** This method is quicker than saving data directly to an Excel file, especially when working with large datasets.
- **Accuracy:** By copying from a text editor, you avoid common formatting issues that can arise when pasting data directly from a browser to Excel. Usage of text editor is crucial as it will help recognise each new line as a new row, if we directly copy-paste the entire output in Excel, it will read it in one single cell.
- **Ease of Data Matching:** Including the portal link in your output simplifies the process of aligning the new data with the existing rows in your Excel sheet.
